# Matrix-vector multiplication via dot products

This notebook walks you through how to implement $ y := A x + y $ that "marches" through the matrix in an alternative way.

## Getting started

We will use some functions that are part of our laff library (of which this function will become a part) as well as some routines from the FLAME API (Application Programming Interface) that allows us to write code that closely resembles how we typeset algorithms using the FLAME notation.  These functions are imported with the "import laff as laff" and "import flame" statements.

## Algorithm that takes dot products

<img src="https://studio.edx.org/c4x/UTAustinX/UT.5.01x/asset/423_Mvmult_n_unb_var1B.png" alt="Alternative matrix-vector multiplication via dot products algorithm" width="50%">

## The routine <br> <code> Mvmult_n_unb_var1B( A, x, y ) </code>

This routine, given $ A \in \mathbb{R}^{n \times n} $, $ x \in \mathbb{R}^n $, and $ y \in \mathbb{R}^n $, computes $ y := A x + y $.  The "_n_" in the title of the routine indicates that this is the "no transpose" matrix-vector multiplication.  The "B" means this is the algorithm that marches through matrices from top-left to bottom-right.

The specific laff functions we will use are 
<ul>
<li> <code> laff.dots( x, y, alpha ) </code> which computes $ \alpha := x^T y + \alpha $.  </li>
</ul>

Use the <a href="https://studio.edx.org/c4x/UTAustinX/UT.5.01x/asset/index.html"> Spark webpage</a> to generate a code skeleton.  (Make sure you adjust the name of the routine.)

In [1]:
import flame
import laff as laff

def Mvmult_n_unb_var1B(A, x, y):

    ATL, ATR, \
    ABL, ABR  = flame.part_2x2(A, \
                               0, 0, 'TL')

    xT, \
    xB  = flame.part_2x1(x, \
                         0, 'TOP')

    yT, \
    yB  = flame.part_2x1(y, \
                         0, 'TOP')

    while ATL.shape[0] < A.shape[0]:

        A00,  a01,     A02,  \
        a10t, alpha11, a12t, \
        A20,  a21,     A22   = flame.repart_2x2_to_3x3(ATL, ATR, \
                                                       ABL, ABR, \
                                                       1, 1, 'BR')

        x0,   \
        chi1, \
        x2    = flame.repart_2x1_to_3x1(xT, \
                                        xB, \
                                        1, 'BOTTOM')

        y0,   \
        psi1, \
        y2    = flame.repart_2x1_to_3x1(yT, \
                                        yB, \
                                        1, 'BOTTOM')

        #------------------------------------------------------------#

        laff.dots(a10t, x0, psi1)
        laff.dots(alpha11, chi1, psi1)
        laff.dots(a12t, x2, psi1)

        #------------------------------------------------------------#

        ATL, ATR, \
        ABL, ABR  = flame.cont_with_3x3_to_2x2(A00,  a01,     A02,  \
                                               a10t, alpha11, a12t, \
                                               A20,  a21,     A22,  \
                                               'TL')

        xT, \
        xB  = flame.cont_with_3x1_to_2x1(x0,   \
                                         chi1, \
                                         x2,   \
                                         'TOP')

        yT, \
        yB  = flame.cont_with_3x1_to_2x1(y0,   \
                                         psi1, \
                                         y2,   \
                                         'TOP')

    flame.merge_2x1(yT, \
                    yB, y)

## Testing

Let's quickly test the routine by creating a 4 x 4 matrix and related vectors, performing the computation.

In [2]:
from numpy import random
from numpy import matrix

A = matrix( random.rand( 4,4 ) )
x = matrix( random.rand( 4,1 ) )
y = matrix( random.rand( 4,1 ) )
yold = matrix( random.rand( 4,1 ) )

print( 'A before =' )
print( A )

print( 'x before =' )
print( x )

print( 'y before =' )
print( y )

A before =
[[0.63559218 0.65629617 0.92758114 0.43865598]
 [0.75142731 0.39028472 0.34946391 0.35045189]
 [0.99595583 0.08263465 0.97955522 0.64500611]
 [0.7758737  0.55082389 0.97195363 0.31203025]]
x before =
[[0.7072026 ]
 [0.71587233]
 [0.05149987]
 [0.47884544]]
y before =
[[0.51408479]
 [0.75478234]
 [0.04052704]
 [0.06018311]]


In [3]:
laff.copy( y, yold )   # save the original vector y

Mvmult_n_unb_var1B( A, x, y )

print( 'y after =' )
print( y )

print( 'y - ( A * x + yold ) = ' )
print( y - ( A * x + yold ) )

y after =
[[1.69122023]
 [1.75139736]
 [1.16333066]
 [1.20267233]]
y - ( A * x + yold ) = 
[[ 2.22044605e-16]
 [ 4.44089210e-16]
 [-2.22044605e-16]
 [-2.22044605e-16]]


Bingo, it seems to work!  (Notice that we are doing floating point computations, which means that due to rounding you may not get an exact "0", but it should be close.)

## Watch your code in action!

Copy and paste the code into <a href="http://edx-org-utaustinx.s3.amazonaws.com/UT501x/PictureFlame/PictureFLAME.html"> PictureFLAME </a>, a webpage where you can watch your routine in action.  Just cut and paste into the box.  

Disclaimer: we implemented a VERY simple interpreter.  If you do something wrong, we cannot guarantee the results.  But if you do it right, you are in for a treat.

If you want to reset the problem, just click in the box into which you pasted the code and hit "next" again.

## Algorithm that uses axpys

<img
src="https://studio.edx.org/c4x/UTAustinX/UT.5.01x/asset/423_Mvmult_n_unb_var2B.png" alt="Alternative matrix-vector multiplication via dot products algorithm" width="50%">

## The routine <br> <code> Mvmult_n_unb_var2B( A, x, y ) </code>

This routine, given $ A \in \mathbb{R}^{n \times n} $, $ x \in \mathbb{R}^n $, and $ y \in \mathbb{R}^n $, computes $ y := A x + y $.  The "_n_" in the name of the routine indicates this is the "no transpose" matrix-vector multiplication.  The "B" means this is the algorithm that marches through matrices from top-left to bottom-right.

The specific laff functions we will use are 
<ul>
<li> <code> laff.axpy( alpha, x, y ) </code> which computes $ y := \alpha x +  y  $.  </li>
</ul>

Use the <a href="https://studio.edx.org/c4x/UTAustinX/UT.5.01x/asset/index.html"> Spark webpage</a> to generate a code skeleton.  (Make sure you adjust the name of the routine.)

In [4]:
#Insert code here...
def Mvmult_n_unb_var2B(A, x, y):

    ATL, ATR, \
    ABL, ABR  = flame.part_2x2(A, \
                               0, 0, 'TL')

    xT, \
    xB  = flame.part_2x1(x, \
                         0, 'TOP')

    yT, \
    yB  = flame.part_2x1(y, \
                         0, 'TOP')

    while ATL.shape[0] < A.shape[0]:

        A00,  a01,     A02,  \
        a10t, alpha11, a12t, \
        A20,  a21,     A22   = flame.repart_2x2_to_3x3(ATL, ATR, \
                                                       ABL, ABR, \
                                                       1, 1, 'BR')

        x0,   \
        chi1, \
        x2    = flame.repart_2x1_to_3x1(xT, \
                                        xB, \
                                        1, 'BOTTOM')

        y0,   \
        psi1, \
        y2    = flame.repart_2x1_to_3x1(yT, \
                                        yB, \
                                        1, 'BOTTOM')

        #------------------------------------------------------------#

        laff.axpy(chi1, a01, y0)
        laff.axpy(chi1, alpha11, psi1)
        laff.axpy(chi1, a21, y2)

        #------------------------------------------------------------#

        ATL, ATR, \
        ABL, ABR  = flame.cont_with_3x3_to_2x2(A00,  a01,     A02,  \
                                               a10t, alpha11, a12t, \
                                               A20,  a21,     A22,  \
                                               'TL')

        xT, \
        xB  = flame.cont_with_3x1_to_2x1(x0,   \
                                         chi1, \
                                         x2,   \
                                         'TOP')

        yT, \
        yB  = flame.cont_with_3x1_to_2x1(y0,   \
                                         psi1, \
                                         y2,   \
                                         'TOP')

    flame.merge_2x1(yT, \
                    yB, y)

## Testing

Let's quickly test the routine by creating a 4 x 4 matrix and related vectors, performing the computation.

In [5]:
from numpy import random
from numpy import matrix

A = matrix( random.rand( 4,4 ) )
x = matrix( random.rand( 4,1 ) )
y = matrix( random.rand( 4,1 ) )
yold = matrix( random.rand( 4,1 ) )

print( 'A before =' )
print( A )

print( 'x before =' )
print( x )

print( 'y before =' )
print( y )

A before =
[[0.5834093  0.39872041 0.46494132 0.27901801]
 [0.64386537 0.30801273 0.46319564 0.75000868]
 [0.37538016 0.76390995 0.43483208 0.27272428]
 [0.139013   0.01769376 0.67471268 0.94006107]]
x before =
[[0.38014322]
 [0.67910146]
 [0.03724552]
 [0.12573779]]
y before =
[[0.44257551]
 [0.93652002]
 [0.18378187]
 [0.19935976]]


In [6]:
laff.copy( y, yold )   # save the original vector y

Mvmult_n_unb_var2B( A, x, y )

print( 'y after =' )
print( y )

print( 'y - ( A * x + yold ) = ' )
print( y - ( A * x + yold ) )

y after =
[[0.9875263 ]
 [1.50200937]
 [0.89573975]
 [0.40755169]]
y - ( A * x + yold ) = 
[[0.00000000e+00]
 [2.22044605e-16]
 [0.00000000e+00]
 [5.55111512e-17]]


Bingo, it seems to work!  (Notice that we are doing floating point computations, which means that due to rounding you may not get an exact "0".)

## Watch your code in action!

Copy and paste the code into <a href="http://edx-org-utaustinx.s3.amazonaws.com/UT501x/PictureFlame/PictureFLAME.html"> PictureFLAME </a>, a webpage where you can watch your routine in action.  Just cut and paste into the box.  

Disclaimer: we implemented a VERY simple interpreter.  If you do something wrong, we cannot guarantee the results.  But if you do it right, you are in for a treat.

If you want to reset the problem, just click in the box into which you pasted the code and hit "next" again.